Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll save SQL files as multiline string variables that they can then use to create the database on their end. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `id` | `serial` | Client ID - primary key | `client_id` |
| `age` | `integer` | Client's age in years | `age` |
| `job` | `text` | Client's type of job | `job` |
| `marital` | `text` | Client's marital status | `marital` | 
| `education` | `text` | Client's level of education | `education` |
| `credit_default` | `boolean` | Whether the client's credit is in default | `credit_default` |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) | `housing` | 
| `loan` | `boolean` | Whether the client has an existing personal loan | `loan` |

<br>

## campaign

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `campaign_id` | `serial` | Campaign ID - primary key | N/A - new column |
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | `campaign` |
| `contact_duration` | `integer` | Last contact duration in seconds | `duration` |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) | `pdays` |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | `previous` |
| `previous_outcome` | `boolean` | Outcome of the previous campaign | `poutcome` |
| `campaign_outcome` | `boolean` | Outcome of the current campaign | `y` |
| `last_contact_date` | `date` | Last date the client was contacted | A combination of `day`, `month`, and the newly created `year` |

<br>

## economics

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) | `emp_var_rate` |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | `cons_price_idx` |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) | `euribor3m` |
| `number_employed` | `float` | Number of employees (quarterly indicator)| `nr_employed` |

In [1]:
import pandas as pd
import numpy as np

# Loading the bank marketing csv into a pandas dataframe
bank_marketing = pd.read_csv('bank_marketing.csv')
print(bank_marketing.head(n=5))

   client_id  age        job  marital    education credit_default housing  \
0          0   56  housemaid  married     basic.4y             no      no   
1          1   57   services  married  high.school        unknown      no   
2          2   37   services  married  high.school             no     yes   
3          3   40     admin.  married     basic.6y             no      no   
4          4   56   services  married  high.school             no      no   

  loan    contact month  ...  campaign  pdays  previous     poutcome  \
0   no  telephone   may  ...         1    999         0  nonexistent   
1   no  telephone   may  ...         1    999         0  nonexistent   
2   no  telephone   may  ...         1    999         0  nonexistent   
3   no  telephone   may  ...         1    999         0  nonexistent   
4  yes  telephone   may  ...         1    999         0  nonexistent   

   emp_var_rate cons_price_idx  cons_conf_idx  euribor3m  nr_employed   y  
0           1.1         93.9

In [2]:
#Splitting the bank marketing dataframe into three dataframes using the information provided

original_client_columns = ['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing', 'loan']

original_campaign_columns = ['client_id', 'campaign', 'duration', 'pdays', 'previous', 'poutcome', 'y', 'day', 'month']

original_economics_columns = ['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor3m', 'nr_employed']

client = bank_marketing[original_client_columns]
campaign = bank_marketing[original_campaign_columns]
economics = bank_marketing[original_economics_columns]

print(client.head(5))
print(campaign.head(5))
print(economics.head(5))

   client_id  age        job  marital    education credit_default housing loan
0          0   56  housemaid  married     basic.4y             no      no   no
1          1   57   services  married  high.school        unknown      no   no
2          2   37   services  married  high.school             no     yes   no
3          3   40     admin.  married     basic.6y             no      no   no
4          4   56   services  married  high.school             no      no  yes
   client_id  campaign  duration  pdays  previous     poutcome   y  day month
0          0         1       261    999         0  nonexistent  no   13   may
1          1         1       149    999         0  nonexistent  no   19   may
2          2         1       226    999         0  nonexistent  no   23   may
3          3         1       151    999         0  nonexistent  no   27   may
4          4         1       307    999         0  nonexistent  no    3   may
   client_id  emp_var_rate  cons_price_idx  euribor3m  nr_

**Renaming specific columns according to the criteria below:**

"client_id" to "id" in **client** (leave as-is in the other subsets);

"duration" to "contact_duration", 
"previous" to "previous_campaign_contacts",
"y" to "campaign_outcome", 
"poutcome" to "previous_outcome", and 
"campaign" to "number_contacts" in **campaign**;

"euribor3m" to "euribor_three_months" and "nr_employed" to "number_employed" in **economics**.

In [3]:
# Renaming columns in three dataframes

client.rename(columns = {'client_id':'id'}, inplace=True)

campaign.rename(columns = {'duration':'contract_duration',                     'previous':'previous_campaign_contracts', 'y':'campaign_outcome',
'poutcome':'previous_outcome', 'campaign':'number_contracts'}, inplace=True)

economics.rename(columns = {'euribor3m':'euribor_three_months', 'nr_employed':'number_employed'}, inplace=True)

# Printing a list of columns for wach dataframe
print(client.keys())
print(campaign.keys())
print(economics.keys())

Index(['id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing',
       'loan'],
      dtype='object')
Index(['client_id', 'number_contracts', 'contract_duration', 'pdays',
       'previous_campaign_contracts', 'previous_outcome', 'campaign_outcome',
       'day', 'month'],
      dtype='object')
Index(['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor_three_months',
       'number_employed'],
      dtype='object')


C:\Users\USER\AppData\Local\Temp\ipykernel_8560\2447106245.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client.rename(columns = {'client_id':'id'}, inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_8560\2447106245.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign.rename(columns = {'duration':'contract_duration',                     'previous':'previous_campaign_contracts', 'y':'campaign_outcome',
C:\Users\USER\AppData\Local\Temp\ipykernel_8560\2447106245.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https: